<a href="https://colab.research.google.com/github/bracealround/Bangla_Newspaper_Scrappers/blob/main/ScrapeBDProtidin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import bs4
import requests
import os
import pandas as pd
import re
from datetime import timedelta, datetime

In [ ]:
def getNewsFromCategoryAndDate(category, date):
  url = f"https://www.bd-pratidin.com/{category}/{date}"
  page = requests.get(url)
  soup = bs4.BeautifulSoup(page.content,"html.parser").find("div",{"class": "container-left-area col-md-9"})
  newsLinks = []

  for a in soup.findAll("a", {"href": re.compile(r"^[a-z-]+\/\d+\/\d+\/\d+\/\d+$")}):
    newsLinks.append(a["href"])

  return newsLinks

In [ ]:
def getNewsFromLink(link):
  url = f"https://www.bd-pratidin.com/{link}"
  page = requests.get(url)
  soup = bs4.BeautifulSoup(page.content, "html.parser")

  title = soup.find("h1", {"class":"post-title"}).text.strip()
  description = soup.find("meta",{"property":"og:description"})["content"].strip()
  category = link.split("/")[0]
  id = int(link.split("/")[4])
  date = "/".join(link.split("/")[1:4])

  articleSoup = soup.find("article")
  article = ""
  for p in articleSoup.findAll("p"):
    article += p.text

  return {"id": id,
          "title": title, 
          "description": description, 
          "category": category,
          "date": date,
          "article": article}

In [ ]:
def getNewsFromDate(date, save=0, verbose=1):
  url = f"https://www.bd-pratidin.com/archive/{date}"
  page = requests.get(url)
  soup = bs4.BeautifulSoup(page.content, "html.parser").find("div", {"class": "container-left-area printversion col-md-9"})
  categories = set()

  for a in soup.findAll("a", {"href": re.compile(r"^[a-z-]+\/\d+\/\d+\/\d+")}):
    categories.add(a["href"].split("/")[0])
  categories = list(categories)

  if verbose==1:
    print(f"There are {len(categories)} categories.")

  news = []
  for category in categories:
    newsLink = getNewsFromCategoryAndDate(category, date)

    if verbose==1:
      print(f"Downloading {len(newsLink)} news from '{category}' category...")

    for link in newsLink:
      news.append(getNewsFromLink(link))

    if verbose==1:
      print(f"Done!")

  news_df = pd.DataFrame(news)

  if save != 0:
    news_df.to_csv(save, index=False)

  return news_df

In [ ]:
def saveNewsFromMultipleDate(days=0, startDate=datetime.today(), verbose=1):
  for day in range(days+1):
    date = (startDate - timedelta(days=day)).strftime("%Y/%m/%d")
    filename = "-".join(date.split("/")) + ".csv"

    if verbose==1:
      print(f"Saving news from {date} in {filename}...")

    getNewsFromDate(date, filename, verbose)

    if verbose==1:
      print(f"Saving news from {date} in {filename} is complete.\n")

In [ ]:


# saveNewsFromMultipleDate(10)

Saving news from 2020/09/02 in 2020-09-02.csv...
There are 11 categories.
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Saving news from 2020/09/02 in 2020-09-02.csv is complete.

Saving news from 2020/09/01 in 2020-09-01.csv...
There are 12 categories.
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Saving news from 2020/09/01 in 2020-09-01.csv is complete.

Saving news from 2020/08/31 in 2020-08-31.csv...
There are 1 categories.
Done!
Saving news from 2020/08/31 in 2020-08-31.csv is complete.

Saving news from 2020/08/30 in 2020-08-30.csv...
There are 12 categories.
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Saving news from 2020/08/30 in 2020-08-30.csv is complete.

Saving news from 2020/08/29 in 2020-08-29.csv...
There are 12 categories.
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Saving news from 2020/08/29 in 2020-08-29.csv is complete.

Saving news from 2020/08/28 in 2020-08-28.c